In [1]:
import json
import random
data = set()

with open("./xxxy.json", "r", encoding="utf-8") as f:
    data1 = json.load(f)
print(len(data1))
with open("./rjxy.json", "r", encoding="utf-8") as f:
    data2 = json.load(f)
with open("./sxytjxy.json", "r", encoding="utf-8") as f:
    data3 = json.load(f)

dd = data1 + data2 + data3
print(len(data1), len(data2), len(data3), len(dd))

for line in dd:
    data.add((line['name'], line['courseName'], line['collegeName'], line['classroom'], line['building']))
print(len(data))


249
249 185 251 685
520


In [2]:


campuses = "呈贡校区"


# 简单问题模板
simple_templates = [
    "{name}老师的联系方式是什么？",
    "介绍一下{name}老师。",
    "{name}老师的职称是什么？",
    "{name}老师在哪个学院？",
    "{name}老师的邮箱是多少？",
    "{course}是谁教的？",
    "{course}的学分是多少？",
    "{classroom}的容量有多大？",
    "{college}的官方网址是什么？"
]

# 两跳问题模板
two_hop_templates = [
    "{name}老师教授的课程有哪些？",
    "{name}老师教授的{course}在哪上课？",
    "{name}老师教授的{course}的上课时间和地点是什么？",
    "{course}的上课地点在哪个校区？",
    "{course}的教学班有多少人？",
    "{classroom}在哪个校区？",
]

# 复杂问题模板
complex_templates = [
    "{name}老师所在学院官网是什么？开设的课程有哪些（列出5个）？",
    "在{building}上课的所有课程中，哪些由{college}开设，且学分超过3？",
    "{college}有哪些老师发表过关于“人工智能”的论文？",
    "同时在{classroom}上课的老师有哪些（列出5个）？他们教授的课程是什么？（列出5个）",
    "找出周一下午在{building}有课且职称是教授的老师（列出5个）"
]
data_list = list(data)

''' {
    "name": "黄光能",
    "courseName": "图说人际关系心理",
    "collegeName": "软件学院",
    "classroom": "格物楼1栋1406",
    "building": "格物楼1栋"
  }'''
# 填充函数
def fill_templates(templates, data_list):
    d = []
    for line in data_list:
        for template in templates:
            filled = template.format(
                name=line[0],
                course=line[1],
                classroom=line[3],
                campus=campuses,
                college=line[2],
                building=line[4]
            )
            d.append({"question": filled})
    return d

data_list = list(data)
# 生成测试数据
test_data = {
    "simple": fill_templates(simple_templates, data_list),
    "two_hop": fill_templates(two_hop_templates, data_list),
    "complex": fill_templates(complex_templates, data_list)
}

# 保存为 JSON 文件
with open("test_questions.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

print("测试问题已保存到 test_q2.json")


测试问题已保存到 test_q2.json


# Merge

In [3]:
with open("./test_questions.json", "r", encoding="utf-8") as f:
    neo4j_qa = json.load(f)
with open('./open.json', "r", encoding="utf-8") as f:
    open_qa = json.load(f)
with open('./vector.json', "r", encoding='utf-8') as f:
    vector_qa = json.load(f)
print(len(open_qa), len(vector_qa), len(neo4j_qa))


50 100 3


In [4]:
print(type(open_qa))
print(open_qa)
print(vector_qa)

<class 'list'>
[{'question': '今天昆明的天气怎么样？'}, {'question': '现在几点了？'}, {'question': '人类是怎么登上月球的？'}, {'question': '如何快速学会Python编程？'}, {'question': '为什么天空是蓝色的？'}, {'question': '水能灭火，那油火为什么不能用水灭？'}, {'question': '1+1为什么等于2？'}, {'question': '你能给我讲一个笑话吗？'}, {'question': '推荐几本入门人工智能的书籍。'}, {'question': '中国的最高山峰是哪座？'}, {'question': '如何预防感冒？'}, {'question': '喝咖啡真的会提神吗？'}, {'question': '今天适合出门散步吗？'}, {'question': '介绍一下人工智能的发展历史。'}, {'question': '你知道地球有多重吗？'}, {'question': '“元宇宙”是什么意思？'}, {'question': '我可以如何提高学习效率？'}, {'question': '今年春节是几号？'}, {'question': '世界上最长的河是哪条？'}, {'question': '你能做些什么？'}, {'question': '大学生应该怎么规划职业发展？'}, {'question': '推荐一部适合周末放松的电影。'}, {'question': '人一天应该喝多少水？'}, {'question': '太阳是怎么发光的？'}, {'question': '狗为什么会摇尾巴？'}, {'question': '如何快速背单词？'}, {'question': '你是谁？'}, {'question': '介绍一下图灵测试。'}, {'question': '为什么会打哈欠？'}, {'question': '人为什么会做梦？'}, {'question': '宇宙有边界吗？'}, {'question': '怎么防止手机上瘾？'}, {'question': '为什么猫喜欢蹭人？'}, {'question': '什么是黑洞？'}, {'question': '可以用一句话鼓励我吗？'}, {'q

In [5]:
import json

with open("./test_questions.json", "r", encoding="utf-8") as f:
    neo4j_qa = json.load(f)
with open('./open.json', "r", encoding="utf-8") as f:
    open_qa = json.load(f)
with open('./vector.json', "r", encoding='utf-8') as f:
    vector_qa = json.load(f)
print(type(neo4j_qa))
simple = neo4j_qa["simple"]
middle = neo4j_qa["two_hop"]
hard = neo4j_qa["complex"]

print(len(simple), len(middle), len(hard))

# 随机选取simple中的1000个问题
import random
simple_sample = random.sample(simple, 300)
middle_sample = random.sample(middle, 100)
hard_sample = random.sample(hard, 100)
print(hard_sample)

<class 'dict'>
4680 3120 2600
[{'question': '在数统学院上课的所有课程中，哪些由数学与统计学院开设，且学分超过3？'}, {'question': '关莉老师所在学院官网是什么？开设的课程有哪些（列出5个）？'}, {'question': '李海老师所在学院官网是什么？开设的课程有哪些（列出5个）？'}, {'question': '同时在力行楼4105（小阶梯教室）上课的老师有哪些（列出5个）？他们教授的课程是什么？（列出5个）'}, {'question': '找出周一下午在信息学院有课且职称是教授的老师（列出5个）'}, {'question': '李成梁老师所在学院官网是什么？开设的课程有哪些（列出5个）？'}, {'question': '同时在文渊楼一楼机房A区上课的老师有哪些（列出5个）？他们教授的课程是什么？（列出5个）'}, {'question': '江华老师所在学院官网是什么？开设的课程有哪些（列出5个）？'}, {'question': '在格物楼1栋上课的所有课程中，哪些由数学与统计学院开设，且学分超过3？'}, {'question': '同时在信息学院2204上课的老师有哪些（列出5个）？他们教授的课程是什么？（列出5个）'}, {'question': '同时在格物楼1栋1405上课的老师有哪些（列出5个）？他们教授的课程是什么？（列出5个）'}, {'question': '找出周一下午在力行楼有课且职称是教授的老师（列出5个）'}, {'question': '在物理学院上课的所有课程中，哪些由信息学院开设，且学分超过3？'}, {'question': '找出周一下午在格物楼2栋有课且职称是教授的老师（列出5个）'}, {'question': '同时在格物楼1栋1505上课的老师有哪些（列出5个）？他们教授的课程是什么？（列出5个）'}, {'question': '信息学院有哪些老师发表过关于“人工智能”的论文？'}, {'question': '找出周一下午在格物楼1栋有课且职称是教授的老师（列出5个）'}, {'question': '软件学院有哪些老师发表过关于“人工智能”的论文？'}, {'question': '李震雄老师所在学院官网是什么？开设的课程有哪些（列出5个）

In [7]:
all_data = []
for q in simple_sample:
    temp = {'question': q['question'], 'intention': 'neo4j', 'level': 'simple', 'answer': ''}
    all_data.append(temp)
for q in middle_sample:
    temp = {'question': q['question'], 'intention': 'neo4j', 'level':'medium', 'answer': ''}
    all_data.append(temp)
for q in hard_sample:
    temp = {'question': q['question'], 'intention': 'neo4j', 'level': 'hard', 'answer': ''}
    all_data.append(temp)
for q in open_qa:
    temp = {'question': q['question'], 'intention': 'open', 'level': 'simple', 'answer': ''}
    all_data.append(temp)
for q in vector_qa:
    temp = {'question': q['question'], 'intention': 'vector', 'level': 'simple', 'answer': ''}
    all_data.append(temp)
s1 = random.sample(middle_sample, 50)
s2 = random.sample(simple_sample, 100)
for q1, q2, q3 in zip(s1, vector_qa, open_qa):
    temp = {'question': q1['question'] + q2['question'] + q3['question'], 'intention': 'neo4j+vector+open', 'level': 'hard', 'answer': ''}
    all_data.append(temp)
for q1, q2 in zip(s2, vector_qa):
    temp = {'question': q1['question'] + q2['question'], 'intention': 'neo4j+vector', 'level': 'hard', 'answer': ''}
    all_data.append(temp)
print(len(all_data))

with open('./test_q4.json', 'w', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)
    



800
